In [2]:
import pandas as pd

In [24]:
df = pd.read_csv('data/oh_county_list.csv')


,County
0,Adams County
1,Allen County
2,Ashland County
3,Ashtabula County
4,Athens County
...,...
83,Washington County
84,Wayne County
85,Williams County
86,Wood County


In [1]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

model.x = pyo.Var([1,2], domain=pyo.NonNegativeReals)

model.OBJ = pyo.Objective(expr = 2*model.x[1] + 3*model.x[2])

model.Constraint1 = pyo.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)